In [1]:
import geopandas as gpd
import pandas as pd
import requests, json
from pyproj import CRS
import ipywidgets as widgets
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


#retrieve data
response= requests.get("http://127.0.0.1:5000/measurements")
rawdata=response.text
data=json.loads(rawdata)
df_measurements=pd.json_normalize(data)
df_measurements.sort_values(by='value')

response1=requests.get("http://127.0.0.1:5000/sensor")
rawdata1=response1.text
df1=json.loads(rawdata1)
df1=pd.json_normalize(df1)


gdf1_sensor = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1.lng, df1.lat))
output_path = 'output/Stations.gpkg'
gdf1_sensor.to_file(output_path, driver='GPKG')
target_crs = CRS.from_epsg(4326) 
gdf1_sensor.crs = target_crs
print(gdf1_sensor.crs)

gdf_sen_meas = gdf1_sensor.merge(df_measurements, on='sensorid', how='inner')

gdf_sen_meas['date'] = pd.to_datetime(gdf_sen_meas['date'], format='%d/%m/%Y %I:%M:%S %p')
gdf_sen_meas['date'] = pd.to_datetime(gdf_sen_meas['date']).dt.dayofweek
gdf_sen_meas.rename(columns={'date':'day'}, inplace=True)

#group "value" by day of week
gdf_sen_meas_days = gdf_sen_meas.groupby('day')['value'].median()

#group "value" by town
gdf_sen_meas_town = gdf_sen_meas.groupby('town')['value'].median()



# Create a Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    dcc.Dropdown(
        id='series-dropdown',
        options=[
            {'label': 'Days', 'value': 'days'},
            {'label': 'Towns', 'value': 'towns'}
        ],
        value='days'  # Default value
    ),
    dcc.Graph(id='bar-plot')
])

# Define the callback to update the graph
@app.callback(
    Output('bar-plot', 'figure'),
    Input('series-dropdown', 'value')
)
def update_graph(selected_series):
    if selected_series == 'days':
        data = gdf_sen_meas_days
    else:
        data = gdf_sen_meas_town
    
    fig = px.bar(x=data.index, y=data.values, labels={'x': 'Category', 'y': 'Value'})
    return fig

# Run the app in Jupyter
app.run_server(mode='inline')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)